In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from json_logic import jsonLogic
from helpers.strings import get_obj_from_path, clean_latin_string
from collections import Counter
from helpers.dates import get_dates_by_frequency
from datetime import timedelta
from helpers.apps.csp import CspEndpoint

## Get Company

In [4]:
company_name = "Eixo SP"

In [5]:
companies = Company.objects.filter(name=company_name)

In [6]:
company = companies.first()

In [7]:
# Get kinds
possible_path_kinds = "occurrencetype__fields__occurrencekind__selectoptions__options"
kinds = get_obj_from_path(company.custom_options, possible_path_kinds)
kind_translation = {
    clean_latin_string(item["name"]).lower(): item["value"] for item in kinds
}

# Get lots
possible_path_lots = "reporting__fields__lot__selectoptions__options"
lots = get_obj_from_path(company.custom_options, possible_path_lots)
lots_translation = {
    item["value"]: item["name"] for item in lots
}

# Indicador 5.1

## Indicador de Conformidade dos Programas de Conservação de Rotina (ICPCR)

Formado por 7 Índices de Conservação de Rotina que compreende os seguintes programas: Pavimento, Faixa de Domínio, Drenagem, Dispositivos de Contenção Viária, Estruturas, Prédios e Pátios e Iluminação.

Fórmula:

𝑁𝑜𝑡𝑎 𝐹𝑖𝑛𝑎𝑙 𝐼𝐶𝑃𝐶𝑅 = (𝑃𝑒𝑠𝑜 𝐼𝐶𝑅𝑃 × 𝑁𝑜𝑡𝑎 𝑑𝑒 𝑃𝑒𝑟𝑓𝑜𝑟𝑚𝑎𝑛𝑐𝑒 𝐼𝐶𝑅𝑃) + (𝑃𝑒𝑠𝑜 𝐼𝐶𝑅𝐹𝐷 × 𝑁𝑜𝑡𝑎 𝑑𝑒 𝑃𝑒𝑟𝑓𝑜𝑟𝑚𝑎𝑛𝑐𝑒 𝐼𝐶𝑅𝐹𝐷) + (𝑃𝑒𝑠𝑜 𝐼𝐶𝑅𝐷 × 𝑁𝑜𝑡𝑎 𝑑𝑒 𝑃𝑒𝑟𝑓𝑜𝑟𝑚𝑎𝑛𝑐𝑒 𝐼𝐶𝑅𝐷) + (𝑃𝑒𝑠𝑜 𝐼𝐶𝑅𝐷𝐶𝑉 × 𝑁𝑜𝑡𝑎 𝑑𝑒 𝑃𝑒𝑟𝑓𝑜𝑟𝑚𝑎𝑛𝑐𝑒 𝐼𝐶𝑅𝐷𝐶𝑉) + (𝑃𝑒𝑠𝑜 𝐼𝐶𝑅𝐸 × 𝑁𝑜𝑡𝑎 𝑑𝑒 𝑃𝑒𝑟𝑓𝑜𝑟𝑚𝑎𝑛𝑐𝑒 𝐼𝐶𝑅𝐸) + (𝑃𝑒𝑠𝑜 𝐼𝐶𝑅𝑃𝑃 × 𝑁𝑜𝑡𝑎 𝑑𝑒 𝑃𝑒𝑟𝑓𝑜𝑟𝑚𝑎𝑛𝑐𝑒 𝐼𝐶𝑅𝑃𝑃) + (𝑃𝑒𝑠𝑜 𝐼𝐶𝑅𝐼 × 𝑁𝑜𝑡𝑎 𝑑𝑒 𝑃𝑒𝑟𝑓𝑜𝑟𝑚𝑎𝑛𝑐𝑒 𝐼𝐶𝑅𝐼)

Para fins de cálculo do ICPCR, os índices possuem os seguintes pesos:
 - Peso ICRP = 0,45
 - Peso ICRFD = 0,20
 - Peso ICRD = 0,10
 - Peso ICRDCV = 0,05
 - Peso ICRE = 0,05
 - Peso ICRPP = 0,05
 - Peso ICRI = 0,10

In [8]:
indicator_number_51 = "5.1"
indicator_number_51_name = "Conservação de Rotina"

icrp_weight = 0.45
icrp_name = "Pavimento"

icrfd_weight = 0.20
icrfd_name = "Faixa de Domínio"

icrd_weight = 0.10
icrd_name = "Drenagem"

icrdcv_weight = 0.05
icrdcv_name = "Dispositivos de Contenção Viária"

icre_weight = 0.05
icre_name = "Estruturas"

icrpp_weight = 0.05
icrpp_name = "Prédios e Pátios"

icri_weight = 0.10
icri_name = "Iluminação"

## ICRP - Índice de Conservação de Rotina de Pavimento

In [9]:
# Calcula a performance dado o ICRP
icrp_performance = {"if" : [
    { ">=": [{"var":"perf"}, 99]}, 1,
    { ">=": [{"var":"perf"}, 98]}, 0.8,
    { ">=": [{"var":"perf"}, 97]}, 0.6,
    { ">=": [{"var":"perf"}, 96]}, 0.4,
    0
]}

In [10]:
# Calcula a conformidade do segmento dado o numero de conformidades
# Se for conforme, retorna 0
# Se for não conforme, retorna 1
# Se for não conforme e agravante, retorna 2
icrp_scoring = {"if" : [
    { ">=": [{"var":"conf"}, 6]}, 2,
    { ">=": [{"var":"conf"}, 3]}, 1,
    0
]}

In [11]:
icrp_occ_kind = ["pavimento flexivel", "pavimento rigido"]
icrp_occ_kind_translated = [kind_translation[item] for item in icrp_occ_kind if item in kind_translation]

In [12]:
icrp_types = [str(item) for item in OccurrenceType.objects.filter(company=company, occurrence_kind__in=icrp_occ_kind_translated).values_list("pk", flat=True).distinct()]

In [13]:
icrp_dict = {
    "name": icrp_name,
    "weight": icrp_weight,
    "performance": icrp_performance,
    "scoring": icrp_scoring,
    "types": icrp_types
}

## ICRFD - Índice de Conservação de Rotina de Faixa de Domínio

In [14]:
# Calcula a performance dado o ICRFD
icrfd_performance = {"if" : [
    { ">=": [{"var":"perf"}, 99]}, 1,
    { ">=": [{"var":"perf"}, 98]}, 0.8,
    { ">=": [{"var":"perf"}, 97]}, 0.6,
    { ">=": [{"var":"perf"}, 96]}, 0.4,
    0
]}

In [15]:
# Calcula a conformidade do segmento dado o numero de conformidades
# Se for conforme, retorna 0
# Se for não conforme, retorna 1
# Se for não conforme e agravante, retorna 2
icrfd_scoring = {"if" : [
    { ">=": [{"var":"conf"}, 8]}, 2,
    { ">=": [{"var":"conf"}, 4]}, 1,
    0
]}

In [16]:
icrfd_occ_kind = ["faixa de dominio"]
icrfd_occ_kind_translated = [kind_translation[item] for item in icrfd_occ_kind if item in kind_translation]

In [17]:
icrfd_types = [str(item) for item in OccurrenceType.objects.filter(company=company, occurrence_kind__in=icrfd_occ_kind_translated).values_list("pk", flat=True).distinct()]

In [18]:
icrfd_dict = {
    "name": icrfd_name,
    "weight": icrfd_weight,
    "performance": icrfd_performance,
    "scoring": icrfd_scoring,
    "types": icrfd_types
}

## ICRD - Índice de Conservação de Rotina de Drenagem

In [19]:
# Calcula a performance dado o ICRD
icrd_performance = {"if" : [
    { ">=": [{"var":"perf"}, 99]}, 1,
    { ">=": [{"var":"perf"}, 98]}, 0.8,
    { ">=": [{"var":"perf"}, 97]}, 0.6,
    { ">=": [{"var":"perf"}, 96]}, 0.4,
    0
]}

In [20]:
# Calcula a conformidade do segmento dado o numero de conformidades
# Se for conforme, retorna 0
# Se for não conforme, retorna 1
# Se for não conforme e agravante, retorna 2
icrd_scoring = {"if" : [
    { ">=": [{"var":"conf"}, 5]}, 2,
    { ">=": [{"var":"conf"}, 3]}, 1,
    0
]}

In [21]:
icrd_occ_kind = ["drenagem"]
icrd_occ_kind_translated = [kind_translation[item] for item in icrd_occ_kind if item in kind_translation]

In [22]:
icrd_types = [str(item) for item in OccurrenceType.objects.filter(company=company, occurrence_kind__in=icrd_occ_kind_translated).values_list("pk", flat=True).distinct()]

In [23]:
icrd_dict = {
    "name": icrd_name,
    "weight": icrd_weight,
    "performance": icrd_performance,
    "scoring": icrd_scoring,
    "types": icrd_types
}

## ICRDCV - Índice de Conservação de Rotina de Dispositivos de Contenção Viária

In [24]:
# Calcula a performance dado o ICRDCV
icrdcv_performance = {"if" : [
    { ">=": [{"var":"perf"}, 99]}, 1,
    { ">=": [{"var":"perf"}, 98]}, 0.8,
    { ">=": [{"var":"perf"}, 97]}, 0.6,
    { ">=": [{"var":"perf"}, 96]}, 0.4,
    0
]}

In [25]:
# Calcula a conformidade do segmento dado o numero de conformidades
# Se for conforme, retorna 0
# Se for não conforme, retorna 1
# Se for não conforme e agravante, retorna 2
icrdcv_scoring = {"if" : [
    { ">=": [{"var":"conf"}, 2]}, 2,
    { ">": [{"var":"conf"}, 0]}, 1,
    0
]}

In [26]:
icrdcv_occ_kind = ["dispositivos de contencao"]
icrdcv_occ_kind_translated = [kind_translation[item] for item in icrdcv_occ_kind if item in kind_translation]

In [27]:
icrdcv_types = [str(item) for item in OccurrenceType.objects.filter(company=company, occurrence_kind__in=icrdcv_occ_kind_translated).values_list("pk", flat=True).distinct()]

In [28]:
icrdcv_dict = {
    "name": icrdcv_name,
    "weight": icrdcv_weight,
    "performance": icrdcv_performance,
    "scoring": icrdcv_scoring,
    "types": icrdcv_types
}

## ICRE - Índice de Conservação de Rotina de Estruturas

In [29]:
# Calcula a performance dado o ICRE
icre_performance = {"if" : [
    { ">=": [{"var":"perf"}, 99]}, 1,
    { ">=": [{"var":"perf"}, 98]}, 0.8,
    { ">=": [{"var":"perf"}, 97]}, 0.6,
    { ">=": [{"var":"perf"}, 96]}, 0.4,
    0
]}

In [30]:
# Calcula a conformidade do segmento dado o numero de conformidades
# Se for conforme, retorna 0
# Se for não conforme, retorna 1
# Se for não conforme e agravante, retorna 2
icre_scoring = {"if" : [
    { ">=": [{"var":"conf"}, 3]}, 2,
    { ">=": [{"var":"conf"}, 2]}, 1,
    0
]}

In [31]:
icre_occ_kind = ["estruturas"]
icre_occ_kind_translated = [kind_translation[item] for item in icre_occ_kind if item in kind_translation]

In [32]:
icre_types = [str(item) for item in OccurrenceType.objects.filter(company=company, occurrence_kind__in=icre_occ_kind_translated).values_list("pk", flat=True).distinct()]

In [33]:
icre_dict = {
    "name": icre_name,
    "weight": icre_weight,
    "performance": icre_performance,
    "scoring": icre_scoring,
    "types": icre_types
}

## ICRPP - Índice de Conservação de Rotina de Prédios e Pátios

In [34]:
# Calcula a performance dado o ICRPP
icrpp_performance = {"if" : [
    { ">=": [{"var":"perf"}, 99]}, 1,
    { ">=": [{"var":"perf"}, 98]}, 0.8,
    { ">=": [{"var":"perf"}, 97]}, 0.6,
    { ">=": [{"var":"perf"}, 96]}, 0.4,
    0
]}

In [35]:
# Calcula a conformidade do segmento dado o numero de conformidades
# Se for conforme, retorna 0
# Se for não conforme, retorna 1
# Se for não conforme e agravante, retorna 2
icrpp_scoring = {"if" : [
    { ">=": [{"var":"conf"}, 10]}, 2,
    { ">=": [{"var":"conf"}, 5]}, 1,
    0
]}

In [36]:
icrpp_occ_kind = ["predios e patios"]
icrpp_occ_kind_translated = [kind_translation[item] for item in icrpp_occ_kind if item in kind_translation]

In [37]:
icrpp_types = [str(item) for item in OccurrenceType.objects.filter(company=company, occurrence_kind__in=icrpp_occ_kind_translated).values_list("pk", flat=True).distinct()]

In [38]:
icrpp_dict = {
    "name": icrpp_name,
    "weight": icrpp_weight,
    "performance": icrpp_performance,
    "scoring": icrpp_scoring,
    "types": icrpp_types
}

## ICRI - Índice de Conservação de Rotina de Iluminação

In [39]:
# Calcula a performance dado o ICRI
icri_performance = {"if" : [
    { ">=": [{"var":"perf"}, 99]}, 1,
    { ">=": [{"var":"perf"}, 98]}, 0.8,
    { ">=": [{"var":"perf"}, 97]}, 0.6,
    { ">=": [{"var":"perf"}, 96]}, 0.4,
    0
]}

In [40]:
# Calcula a conformidade do segmento dado o numero de conformidades
# Se for conforme, retorna 0
# Se for não conforme, retorna 1
# Se for não conforme e agravante, retorna 2
icri_scoring = {"if" : [
    { ">=": [{"var":"conf"}, 2]}, 2,
    { ">": [{"var":"conf"}, 0]}, 1,
    0
]}

In [41]:
icri_occ_kind = ["iluminacao"]
icri_occ_kind_translated = [kind_translation[item] for item in icri_occ_kind if item in kind_translation]

In [42]:
icri_types = [str(item) for item in OccurrenceType.objects.filter(company=company, occurrence_kind__in=icri_occ_kind_translated).values_list("pk", flat=True).distinct()]

In [43]:
icri_dict = {
    "name": icri_name,
    "weight": icri_weight,
    "performance": icri_performance,
    "scoring": icri_scoring,
    "types": icri_types
}

# Indicador 8.1

## Indicador de Integridade e Conservação da Sinalização Vertical (IICSV)

In [44]:
indicator_number_81 = "8.1"
indicator_number_81_name = "Sinalização Vertical"
iicsv_name = "Sinalização Vertical"

In [45]:
# Salvar occurrence_type para o inventorio de Placas
inventory_occ_kind = ["inventario"]
inventory_occ_kind_translated = [kind_translation[item] for item in inventory_occ_kind if item in kind_translation]
inventory_types_qs = OccurrenceType.objects.filter(
    company=company,
    occurrence_kind__in=inventory_occ_kind_translated,
    name__icontains="placa"
)
inventory_types = [str(item) for item in inventory_types_qs.values_list("pk", flat=True).distinct()]

In [46]:
# Calcula a performance dado o IICSV
iicsv_performance = {"if" : [
    { ">=": [{"var":"perf"}, 99]}, 1,
    { ">=": [{"var":"perf"}, 97]}, 0.8,
    { ">=": [{"var":"perf"}, 95]}, 0.6,
    { ">=": [{"var":"perf"}, 93]}, 0.4,
    0
]}

In [47]:
# Calcula a conformidade do segmento dado o numero de conformidades
# Se for conforme, retorna 0
# Se for não conforme, retorna 1
iicsv_scoring = {"if" : [
    { ">=": [{"var":"conf"}, 1]}, {"var":"conf"},
    0
]}

In [48]:
iicsv_occ_kind = ["sinalizacao vertical"]
iicsv_occ_kind_translated = [kind_translation[item] for item in iicsv_occ_kind if item in kind_translation]

In [49]:
iicsv_occ_kind_additional = ["dispositivos auxiliares"]
iicsv_occ_kind_translated_additional = [kind_translation[item] for item in iicsv_occ_kind_additional if item in kind_translation]

In [50]:
iicsv_types_qs_first = OccurrenceType.objects.filter(
    company=company,
    occurrence_kind__in=iicsv_occ_kind_translated
)

iicsv_types_qs_additional = OccurrenceType.objects.filter(
    company=company,
    occurrence_kind__in=iicsv_occ_kind_translated_additional
).filter(
    name__icontains="alerta"
)

iicsv_types_qs = OccurrenceType.objects.filter(Q(pk__in=iicsv_types_qs_first) | Q(pk__in=iicsv_types_qs_additional)).distinct()

In [51]:
iicsv_types = [str(item) for item in iicsv_types_qs.values_list("pk", flat=True).distinct()]

In [52]:
iicsv_dict = {
    "name": iicsv_name,
    "weight": 1,
    "performance": iicsv_performance,
    "scoring": iicsv_scoring,
    "types": iicsv_types
}

# Indicador 8.2

## Indicador de Integridade e Conservação da Sinalização Horizontal (IICSH)

In [53]:
indicator_number_82 = "8.2"
indicator_number_82_name = "Sinalização Horizontal"
iicsh_name = "Sinalização Horizontal"

In [54]:
# Calcula a performance dado o IICSH
iicsh_performance = {"if" : [
    { ">=": [{"var":"perf"}, 97]}, 1,
    { ">=": [{"var":"perf"}, 95]}, 0.8,
    { ">=": [{"var":"perf"}, 93]}, 0.6,
    { ">=": [{"var":"perf"}, 90]}, 0.4,
    0
]}

In [55]:
# Calcula a conformidade do segmento dado o numero de conformidades
# Se for conforme, retorna 0
# Se for não conforme, retorna 1
iicsh_scoring = {"if" : [
    { ">=": [{"var":"conf"}, 1]}, 1,
    0
]}

In [56]:
iicsh_occ_kind = ["sinalizacao horizontal"]
iicsh_occ_kind_translated = [kind_translation[item] for item in iicsh_occ_kind if item in kind_translation]

In [57]:
iicsh_occ_kind_additional = ["dispositivos auxiliares"]
iicsh_occ_kind_translated_additional = [kind_translation[item] for item in iicsh_occ_kind_additional if item in kind_translation]

In [58]:
iicsh_types_qs_first = OccurrenceType.objects.filter(
    company=company,
    occurrence_kind__in=iicsh_occ_kind_translated
)

iicsh_types_qs_additional = OccurrenceType.objects.filter(
    company=company,
    occurrence_kind__in=iicsh_occ_kind_translated_additional
).filter(
    name__icontains="tachas"
)

iicsh_types_qs = OccurrenceType.objects.filter(Q(pk__in=iicsh_types_qs_first) | Q(pk__in=iicsh_types_qs_additional)).distinct()

In [59]:
iicsh_types = [str(item) for item in iicsh_types_qs.values_list("pk", flat=True).distinct()]

In [60]:
iicsh_dict = {
    "name": iicsh_name,
    "weight": 1,
    "performance": iicsh_performance,
    "scoring": iicsh_scoring,
    "types": iicsh_types
}

# Indicador 3.1

## Indicador Socioambiental: Recuperação de Não Conformidades Ambientais (ISA)

In [61]:
indicator_number_31 = "3.1"
indicator_number_31_name = "Indicador Socioambiental"
isa_name = "Recuperação de Não Conformidades Ambientais"

In [62]:
# Calcula a performance dado o ISA
isa_performance = {"if" : [
    { ">=": [{"var":"perf"}, 70]}, 1,
    { ">=": [{"var":"perf"}, 60]}, 0.75,
    { ">=": [{"var":"perf"}, 50]}, 0.5,
    0
]}

In [63]:
# Calcula a conformidade do segmento dado o numero de conformidades
# Se for conforme, retorna 0
# Se for não conforme, retorna 1
isa_scoring = {"if" : [
    { ">=": [{"var":"conf"}, 1]}, 1,
    0
]}

In [64]:
isa_occ_kind = ["meio ambiente"]
isa_occ_kind_translated = [kind_translation[item] for item in isa_occ_kind if item in kind_translation]

In [65]:
isa_types = [str(item) for item in OccurrenceType.objects.filter(company=company, occurrence_kind__in=isa_occ_kind_translated).values_list("pk", flat=True).distinct()]

In [66]:
isa_dict = {
    "name": isa_name,
    "weight": 1,
    "performance": isa_performance,
    "scoring": isa_scoring,
    "types": isa_types
}

## Salvar dados no metadata da Company

In [67]:
inspect_occ_kind = ["fiscalizacao"]
inspect_occ_kind_translated = [kind_translation[item] for item in inspect_occ_kind if item in kind_translation]

In [68]:
inspect_types = [str(item) for item in OccurrenceType.objects.filter(company=company, occurrence_kind__in=inspect_occ_kind_translated).values_list("pk", flat=True).distinct()]

In [69]:
company.metadata["csp"] = {
    "topics": {
        indicator_number_51 : {
            "name": indicator_number_51_name,
            "type": "segment",
            "rules": {
                "ICRP": icrp_dict,
                "ICRFD": icrfd_dict,
                "ICRD": icrd_dict,
                "ICRDCV": icrdcv_dict,
                "ICRE": icre_dict,
                "ICRPP": icrpp_dict,
                "ICRI": icri_dict,
            }
        },
        indicator_number_81 : {
            "name": indicator_number_81_name,
            "type": "inventory",
            "type_ids": inventory_types,
            "rules": {
                "IICSV": iicsv_dict
            }
        },
        indicator_number_82 : {
            "name": indicator_number_82_name,
            "type": "lane_segment",
            "rules": {
                "IICSH": iicsh_dict
            }
        },
        indicator_number_31 : {
            "name": indicator_number_31_name,
            "rules": {
                "ISA": isa_dict
            }
        }
    },
    "lane_types": {
        "Simples": "1",
        "Dupla": "2"
    },
    "lane_systems": {
        "Existente": "1",
        "Remanescente": "2"
    },
    "inspect_types": inspect_types
}

In [70]:
company.save()